Notebook of matrices simulation data 

In [1]:
import numpy as np
from scipy.linalg import logm, sqrtm, expm
from scipy.stats import norm

def gendata_cov(n, p, rho, non_ellip, model, eps):
    # Check input validity
    if p <= 5:
        raise ValueError('Dimension p must be greater or equal to 6')

    # Generate AR1(rho) predictors or standard normal predictors
    if rho > 0:
        X = np.zeros((n, p))
        X[:, 0] = np.random.randn(n)
        for i in range(1, p):
            X[:, i] = rho * X[:, i - 1] + np.sqrt(1 - rho**2) * np.random.randn(n)
    else:
        X = np.random.randn(n, p)

    # Generate non-elliptical predictors if non_ellip != 0
    if non_ellip == 1:
        X[:, 0] = np.sin(X[:, 0])
        X[:, 1] = np.abs(X[:, 1])
    elif non_ellip == 2:
        X[:, 0] = np.abs(X[:, 2] + X[:, 3]) + np.random.randn(n)
        X[:, 1] = np.sqrt(np.abs(X[:, 2] + X[:, 3])) + np.random.randn(n)
    elif non_ellip == -1:
        X = norm.cdf(X)

    # Initialize dx to ensure it's reset each time
    dx = None

    # Define model-specific parameters and transformations
    if model == 'model1':
        d = 2
        def dFun(t1):
            return np.array([[1, 0.8*np.cos(1*t1)],
                             [0.8*np.cos(1*t1), 1]])
        beta_1 = np.array([1, 1] + [0] * (p - 2))
        z1 = X @ beta_1
        dx = np.array([dFun(z) for z in z1])
    elif model == 'model2':
        d = 3
        def dFun(t1, t2):
            return np.array([[1, 0.8 * np.cos(1*t1), 0.8 * np.sin(t2)],
                             [0.8 * np.cos(1*t1), 1, 0.8 * np.cos(1*t1)],
                             [0.8 * np.sin(t2), 0.8 * np.cos(1*t1), 1]])
        beta_1 = np.array([1, 1] + [0] * (p - 2))
        beta_2 = np.array([0] * (p - 2) + [1, 1])
        z1 = X @ beta_1
        z2 = X @ beta_2
        dx = np.array([dFun(z1i, z2i) for z1i, z2i in zip(z1, z2)])
    else:
        raise ValueError(f"Invalid model: {model}. Supported models are 'model1' and 'model2'.")

    if dx is None:
        raise ValueError("Model must be either 'model1' or 'model2'.")

    # Generate Y based on log-normal error
    def geny(dx):
        return np.round((expm(smvnormal(d, logm(dx), eps * np.eye(d))) +
                         expm(smvnormal(d, logm(dx), eps * np.eye(d)))) / 2, 7)
    Y = np.array([geny(dxi) for dxi in dx])

    return X, Y






def smvnormal(d, M=None, Sigma=None):
    """Symmetric matrix variate normal distribution."""
    if M is None:
        M = np.zeros((d, d))
    if Sigma is None:
        Sigma = np.eye(d)
    if np.linalg.norm(M - M.T, 'fro') > 1e-8:
        raise ValueError('Mean covariate must be symmetric')
    if np.any(np.linalg.eigvals(Sigma) < 0):
        raise ValueError('Covariance matrix must be symmetric positive definite')
    
    x = np.zeros((d, d))
    x[np.diag_indices(d)] = np.random.randn(d)
    indices = np.tril_indices(d, -1)
    x[indices] = np.random.randn(len(indices[0])) / np.sqrt(2)
    x = x + x.T - np.diag(np.diag(x))
    G = sqrtm(Sigma)
    return G.T @ x @ G + M


import numpy as np
from scipy.linalg import norm as linalg_norm
from sklearn.covariance import GraphicalLassoCV
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from numpy.linalg import pinv,inv
from numpy.linalg import LinAlgError




# Parameters
n = 200
p = 10
d_0 = 2
rho = 0
non_ellip = 0
num_repeats = 10
metric = "Frobenius"
model = 'model2'
eps = 0.5
neigh = False
beta_1 = np.concatenate([np.array([1, 1]), np.zeros(p - 2)])
beta_2 = np.concatenate([np.zeros(p - 2), np.array([1, 1])])
beta = np.vstack([beta_1,beta_2])
beta_true = beta.T

# To store errors across all simulations
errorsG = np.ones(num_repeats)
errorsD = np.ones(num_repeats)
errorsF = np.ones(num_repeats)

# Repeat simulation num_repeats times
for repeat in range(num_repeats):
    X, Y = gendata_cov2(n, p, rho, non_ellip, model, eps)
    Y = np.real(Y)
    
    Nb = []

    if not neigh:
        # Neighborhood is unknown
        graphical_lasso_model = GraphicalLassoCV()
        graphical_lasso_model.fit(X)
        omega = graphical_lasso_model.precision_
        np.where(np.sum(omega != 0, axis=1) > 1)[0]

    for j in range(p):
        Ni = (np.nonzero(omega[j, :])[0]).tolist()
        Nb.append(Ni)

    # Gram matrix calculations
    ygram = gram_matrix(Y, 1000)
    beta_gwire, _ = gwire_cv(X, Y, Nb, metric, d_0, fold=5)
    b1 = beta_gwire
    ygram2 = gram_matrix(Y, 1)
    beta_fopg = FOPG(X, ygram2, d_0)
    b3 = beta_fopg
    initial = beta_fopg
    beta_dcov, _, _ = MAVE1(X.T, ygram, initial)
    
    b2 = beta_dcov

    # Compute errors
    error2 = linalg_norm(b2@pinv(b2.T@b2)@b2.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsD[repeat] =error2

    error3 = linalg_norm(b3@pinv(b3.T@b3)@b3.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsF[repeat] = error3

    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    errorsG[repeat] = error1

  

# Compute means and standard deviations
mean_errorG = np.mean(errorsG)
sd_errorG = np.std(errorsG)

mean_errorD = np.mean(errorsD)
sd_errorD = np.std(errorsD)

mean_errorF = np.mean(errorsF)
sd_errorF = np.std(errorsF)

# Print the results
print('Model2-a,(200,10)')
print(f"Error  (G-WIRE): Mean = {mean_errorG:.4f}, SD = {sd_errorG:.4f}")
print(f"Error  (DCOV): Mean = {mean_errorD:.4f}, SD = {sd_errorD:.4f}")
print(f"Error  (FOPG): Mean = {mean_errorF:.4f}, SD = {sd_errorF:.4f}")


In [2]:
# rho(x) = cos(2*beta^T@X)

import numpy as np
from scipy.linalg import norm as linalg_norm
from sklearn.covariance import GraphicalLassoCV
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from numpy.linalg import inv
from numpy.linalg import LinAlgError
import time



# Parameters
n = 200
p = 10
d_0 = 1
rho = 0
non_ellip = 0
num_repeats = 1
metric = "Frobenius"
model = 'model1'
eps = 0.5
neigh = False
beta_1 = np.concatenate([np.array([1, 1]), np.zeros(p - 2)])
beta_true = beta_1.reshape((p, 1))

# To store errors across all simulations
errorsG = np.ones(num_repeats)
errorsD = np.ones(num_repeats)
errorsF = np.ones(num_repeats)

# Repeat simulation num_repeats times
for repeat in range(num_repeats):
    X, Y = gendata_cov(n, p, rho, non_ellip, model, eps)
    Nb = []

    if not neigh:
        # Neighborhood is unknown
        graphical_lasso_model = GraphicalLassoCV()
        graphical_lasso_model.fit(X)
        omega = graphical_lasso_model.precision_
        np.where(np.sum(omega != 0, axis=1) > 1)[0]

    for j in range(p):
        Ni = (np.nonzero(omega[j, :])[0]).tolist()
        Nb.append(Ni)

    # Gram matrix calculations
    ygram = gram_matrix(Y, 1000)
    start_time = time.time()
    beta_gwire, _ = gwire_cv(X, Y, Nb, metric, d_0, fold=5)
    gwire_time = time.time() - start_time
    print(f"GWIRE time: {gwire_time:.4f} seconds")
    b1 = beta_gwire
    ygram2 = gram_matrix(Y, 1)
    start_time = time.time()
    beta_fopg = FOPG(X, ygram2, d_0)
    fopg_time = time.time() - start_time
    print(f"FOPG time: {fopg_time:.4f} seconds")
    b3 = beta_fopg
    initial = beta_fopg
    start_time = time.time()
    beta_dcov, _, _ = MAVE1(X.T, ygram, initial)
    mave1_time = time.time() - start_time
    print(f"MAVE1 time: {mave1_time:.4f} seconds")
    
    b2 = beta_dcov

    # Compute errors
    error2 = linalg_norm(b2@inv(b2.T@b2)@b2.T-beta_true@inv(beta_true.T@beta_true)@beta_true.T)
    errorsD[repeat] =error2

    error3 = linalg_norm(b3@inv(b3.T@b3)@b3.T-beta_true@inv(beta_true.T@beta_true)@beta_true.T)
    errorsF[repeat] = error3

    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    errorsG[repeat] = error1
  

# Compute means and standard deviations
mean_errorG = np.mean(errorsG)
sd_errorG = np.std(errorsG)

mean_errorD = np.mean(errorsD)
sd_errorD = np.std(errorsD)

mean_errorF = np.mean(errorsF)
sd_errorF = np.std(errorsF)

# Print the results
print('model1-a, (200,10)')
print(f"Error  (G-WIRE): Mean = {mean_errorG:.4f}, SD = {sd_errorG:.4f}")
print(f"Error  (DCOV): Mean = {mean_errorD:.4f}, SD = {sd_errorD:.4f}")
print(f"Error  (FOPG): Mean = {mean_errorF:.4f}, SD = {sd_errorF:.4f}")


GWIRE time: 4.9184 seconds
FOPG time: 0.2086 seconds
MAVE1 time: 0.0159 seconds
model1-a, (200,10)
Error  (G-WIRE): Mean = 1.0331, SD = 0.0000
Error  (DCOV): Mean = 0.1989, SD = 0.0000
Error  (FOPG): Mean = 0.1639, SD = 0.0000


In [3]:
# rho(x) = cos(2*beta^T@X)

import numpy as np
from scipy.linalg import norm as linalg_norm
from sklearn.covariance import GraphicalLassoCV
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from numpy.linalg import pinv



# Parameters
n = 400
p = 20
d_0 = 1
rho = 0
non_ellip = 0
num_repeats = 100
metric = "Frobenius"
model = 'model1'
eps = 0.5
neigh = False
beta_1 = np.concatenate([np.array([1, 1]), np.zeros(p - 2)])
beta_true = beta_1.reshape((p, 1))

# To store errors across all simulations
errorsG = np.ones(num_repeats)
errorsD = np.ones(num_repeats)
errorsF = np.ones(num_repeats)
Time1 = np.zeros(num_repeats)
Time2 = np.zeros(num_repeats)
Time3 = np.zeros(num_repeats)


# Repeat simulation num_repeats times
for repeat in range(num_repeats):
    X, Y = gendata_cov(n, p, rho, non_ellip, model, eps)
    Nb = []

    if not neigh:
        # Neighborhood is unknown
        graphical_lasso_model = GraphicalLassoCV()
        graphical_lasso_model.fit(X)
        omega = graphical_lasso_model.precision_
        np.where(np.sum(omega != 0, axis=1) > 1)[0]

    for j in range(p):
        Ni = (np.nonzero(omega[j, :])[0]).tolist()
        Nb.append(Ni)

    # Gram matrix calculations
    ygram = gram_matrix(Y, 1000)
    start_time = time.time()
    beta_gwire, _ = gwire_cv(X, Y, Nb, metric, d_0, fold=5)
    gwire_time = time.time() - start_time
    Time1[repeat] = gwire_time
    b1 = beta_gwire
    ygram2 = gram_matrix(Y, 1)
    start_time = time.time()
    beta_fopg = FOPG(X, ygram2, d_0)
    fopg_time = time.time() - start_time
    Time2[repeat] = fopg_time
    b3 = beta_fopg
    initial = beta_fopg
    start_time = time.time()
    beta_dcov, _, _ = MAVE1(X.T, ygram, initial)
    mave1_time = time.time() - start_time
    Time3[repeat] = mave1_time
    
    b2 = beta_dcov

    # Compute errors
    error2 = linalg_norm(b2@pinv(b2.T@b2)@b2.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsD[repeat] =error2

    error3 = linalg_norm(b3@pinv(b3.T@b3)@b3.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsF[repeat] = error3

    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    errorsG[repeat] = error1

  

# Compute means and standard deviations
mean_errorG = np.mean(errorsG)
sd_errorG = np.std(errorsG)

mean_errorD = np.mean(errorsD)
sd_errorD = np.std(errorsD)

mean_errorF = np.mean(errorsF)
sd_errorF = np.std(errorsF)

meantimeG = np.mean(Time1)
sdtimeG = np.std(Time1)
meantimeF = np.mean(Time2)
sdtimeF = np.std(Time2)
meantimeD = np.mean(Time3)
sdtimeD = np.std(Time3)
print("GWIRE mean Time:", meantimeG)
print("GWIRE std of Time:", sdtimeG)
print("DCOV mean Time",meantimeD)
print("DCOV std of Time:", sdtimeD)
print("FOPG mean Time:",meantimeF)
print("FOPG std of Time:",sdtimeF)

# Print the results
print('model1-a, (400,20)')
print(f"Error  (G-WIRE): Mean = {mean_errorG:.4f}, SD = {sd_errorG:.4f}")
print(f"Error  (DCOV): Mean = {mean_errorD:.4f}, SD = {sd_errorD:.4f}")
print(f"Error  (FOPG): Mean = {mean_errorF:.4f}, SD = {sd_errorF:.4f}")


GWIRE mean Time: 5.414922692775726
GWIRE std of Time: 2.366642544149801
DCOV mean Time 0.1552799868583679
DCOV std of Time: 0.12142796449230617
FOPG mean Time: 2.76433851480484
FOPG std of Time: 0.10245350139979291
model1-a, (400,20)
Error  (G-WIRE): Mean = nan, SD = nan
Error  (DCOV): Mean = 0.3552, SD = 0.4335
Error  (FOPG): Mean = 0.5542, SD = 0.4943


In [4]:
import numpy as np
import scipy.linalg as la
from scipy.linalg import norm as linalg_norm
from scipy.stats import norm as stats_norm, gamma
from sklearn.covariance import GraphicalLassoCV
from scipy import stats
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import pinv




# Parameters
n = 200
p = 10
d_0 = 1
rho = 0.5
non_ellip = 1
num_repeats = 1
metric = "Frobenius"
model = 'model1'
eps = 0.5
neigh = False
beta_1 = np.concatenate([np.array([1, 1]), np.zeros(p - 2)])
beta_true = beta_1.reshape((p, 1))

# To store errors across all simulations
errorsG = np.ones(num_repeats)
errorsD = np.ones(num_repeats)
errorsF = np.ones(num_repeats)

# Repeat simulation num_repeats times
for repeat in range(num_repeats):
    X, Y = gendata_cov(n, p, rho, non_ellip, model, eps)
    
    Nb = []

    if not neigh:
        # Neighborhood is unknown
        graphical_lasso_model = GraphicalLassoCV()
        graphical_lasso_model.fit(X)
        omega = graphical_lasso_model.precision_
        np.where(np.sum(omega != 0, axis=1) > 1)[0]

    for j in range(p):
        Ni = (np.nonzero(omega[j, :])[0]).tolist()
        Nb.append(Ni)

    # Gram matrix calculations
    ygram = gram_matrix(Y, 1000)
    start_time = time.time()
    beta_gwire, _ = gwire_cv(X, Y, Nb, metric, d_0, fold=5)
    gwire_time = time.time() - start_time
    print(f"GWIRE time: {gwire_time:.4f} seconds")
    b1 = beta_gwire
    ygram2 = gram_matrix(Y, 1)
    start_time = time.time()
    beta_fopg = FOPG(X, ygram2, d_0)
    fopg_time = time.time() - start_time
    print(f"FOPG time: {fopg_time:.4f} seconds")
    b3 = beta_fopg
    initial = beta_fopg
    start_time = time.time()
    beta_dcov, _, _ = MAVE1(X.T, ygram, initial)
    mave1_time = time.time() - start_time
    print(f"MAVE1 time: {mave1_time:.4f} seconds")
    
    b2 = beta_dcov

    # Compute errors
    error2 = linalg_norm(b2@pinv(b2.T@b2)@b2.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsD[repeat] =error2

    error3 = linalg_norm(b3@pinv(b3.T@b3)@b3.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsF[repeat] = error3

    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    errorsG[repeat] = error1

  

# Compute means and standard deviations
mean_errorG = np.mean(errorsG)
sd_errorG = np.std(errorsG)

mean_errorD = np.mean(errorsD)
sd_errorD = np.std(errorsD)

mean_errorF = np.mean(errorsF)
sd_errorF = np.std(errorsF)

# Print the results
print('Model1-b,(200,10)')
print(f"Error  (G-WIRE): Mean = {mean_errorG:.4f}, SD = {sd_errorG:.4f}")
print(f"Error  (DCOV): Mean = {mean_errorD:.4f}, SD = {sd_errorD:.4f}")
print(f"Error  (FOPG): Mean = {mean_errorF:.4f}, SD = {sd_errorF:.4f}")


GWIRE time: 5.7730 seconds
FOPG time: 0.2016 seconds
MAVE1 time: 0.0109 seconds
Model1-b,(200,10)
Error  (G-WIRE): Mean = 0.2701, SD = 0.0000
Error  (DCOV): Mean = 0.2133, SD = 0.0000
Error  (FOPG): Mean = 0.4054, SD = 0.0000


In [5]:
import numpy as np
import scipy.linalg as la
from scipy.linalg import norm as linalg_norm
from scipy.stats import norm as stats_norm, gamma
from sklearn.covariance import GraphicalLassoCV
from scipy import stats
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import pinv




# Parameters
n = 400
p = 20
d_0 = 1
rho = 0.5
non_ellip = 1
num_repeats = 100
metric = "Frobenius"
model = 'model1'
eps = 0.5
neigh = False
beta_1 = np.concatenate([np.array([1, 1]), np.zeros(p - 2)])
beta_true = beta_1.reshape((p, 1))

# To store errors across all simulations
errorsG = np.ones(num_repeats)
errorsD = np.ones(num_repeats)
errorsF = np.ones(num_repeats)
Time1 = np.zeros(num_repeats)
Time2 = np.zeros(num_repeats)
Time3 = np.zeros(num_repeats)

# Repeat simulation num_repeats times
for repeat in range(num_repeats):
    X, Y = gendata_cov(n, p, rho, non_ellip, model, eps)
    
    Nb = []

    if not neigh:
        # Neighborhood is unknown
        graphical_lasso_model = GraphicalLassoCV()
        graphical_lasso_model.fit(X)
        omega = graphical_lasso_model.precision_
        np.where(np.sum(omega != 0, axis=1) > 1)[0]

    for j in range(p):
        Ni = (np.nonzero(omega[j, :])[0]).tolist()
        Nb.append(Ni)

    # Gram matrix calculations
    ygram = gram_matrix(Y, 1000)
    start_time = time.time()
    beta_gwire, _ = gwire_cv(X, Y, Nb, metric, d_0, fold=5)
    gwire_time = time.time() - start_time
    Time1[repeat] = gwire_time
    b1 = beta_gwire
    ygram2 = gram_matrix(Y, 1)
    start_time = time.time()
    beta_fopg = FOPG(X, ygram2, d_0)
    fopg_time = time.time() - start_time
    Time2[repeat] = fopg_time
    b3 = beta_fopg
    initial = beta_fopg
    start_time = time.time()
    beta_dcov, _, _ = MAVE1(X.T, ygram, initial)
    mave1_time = time.time() - start_time
    Time3[repeat] = mave1_time
    
    b2 = beta_dcov

    # Compute errors
    error2 = linalg_norm(b2@pinv(b2.T@b2)@b2.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsD[repeat] =error2

    error3 = linalg_norm(b3@pinv(b3.T@b3)@b3.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsF[repeat] = error3

    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    errorsG[repeat] = error1

  

# Compute means and standard deviations
mean_errorG = np.mean(errorsG)
sd_errorG = np.std(errorsG)

mean_errorD = np.mean(errorsD)
sd_errorD = np.std(errorsD)

mean_errorF = np.mean(errorsF)
sd_errorF = np.std(errorsF)

meantimeG = np.mean(Time1)
sdtimeG = np.std(Time1)
meantimeF = np.mean(Time2)
sdtimeF = np.std(Time2)
meantimeD = np.mean(Time3)
sdtimeD = np.std(Time3)
print("GWIRE mean Time:", meantimeG)
print("GWIRE std of Time:", sdtimeG)
print("DCOV mean Time",meantimeD)
print("DCOV std of Time:", sdtimeD)
print("FOPG mean Time:",meantimeF)
print("FOPG std of Time:",sdtimeF)

# Print the results
print('Model1-b,(400,20)')
print(f"Error  (G-WIRE): Mean = {mean_errorG:.4f}, SD = {sd_errorG:.4f}")
print(f"Error  (DCOV): Mean = {mean_errorD:.4f}, SD = {sd_errorD:.4f}")
print(f"Error  (FOPG): Mean = {mean_errorF:.4f}, SD = {sd_errorF:.4f}")


GWIRE mean Time: 15.81461323261261
GWIRE std of Time: 0.8806323386906417
DCOV mean Time 0.08509934425354004
DCOV std of Time: 0.01590437615205266
FOPG mean Time: 2.838614661693573
FOPG std of Time: 0.13466814943861077
Model1-b,(400,20)
Error  (G-WIRE): Mean = 0.1821, SD = 0.0461
Error  (DCOV): Mean = 0.2209, SD = 0.0479
Error  (FOPG): Mean = 0.2254, SD = 0.0485


In [6]:
import numpy as np
from scipy.linalg import norm as linalg_norm
from sklearn.covariance import GraphicalLassoCV
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from numpy.linalg import pinv




# Parameters
n = 200
p = 10
d_0 = 2
rho = 0
non_ellip = 0
num_repeats = 1
metric = "Frobenius"
model = 'model2'
eps = 0.5
neigh = False
beta_1 = np.concatenate([np.array([1, 1]), np.zeros(p - 2)])
beta_2 = np.concatenate([np.zeros(p - 2), np.array([1, 1])])
beta = np.vstack([beta_1,beta_2])
beta_true = beta.T

# To store errors across all simulations
errorsG = np.ones(num_repeats)
errorsD = np.ones(num_repeats)
errorsF = np.ones(num_repeats)

# Repeat simulation num_repeats times
for repeat in range(num_repeats):
    X, Y = gendata_cov(n, p, rho, non_ellip, model, eps)
    Y = np.real(Y)
    
    Nb = []

    if not neigh:
        # Neighborhood is unknown
        graphical_lasso_model = GraphicalLassoCV()
        graphical_lasso_model.fit(X)
        omega = graphical_lasso_model.precision_
        np.where(np.sum(omega != 0, axis=1) > 1)[0]

    for j in range(p):
        Ni = (np.nonzero(omega[j, :])[0]).tolist()
        Nb.append(Ni)

    # Gram matrix calculations
    ygram = gram_matrix(Y, 1000)
    start_time = time.time()
    beta_gwire, _ = gwire_cv(X, Y, Nb, metric, d_0, fold=5)
    gwire_time = time.time() - start_time
    print(f"GWIRE time: {gwire_time:.4f} seconds")
    b1 = beta_gwire
    ygram2 = gram_matrix(Y, 1)
    start_time = time.time()
    beta_fopg = FOPG(X, ygram2, d_0)
    fopg_time = time.time() - start_time
    print(f"FOPG time: {fopg_time:.4f} seconds")
    b3 = beta_fopg
    initial = beta_fopg
    start_time = time.time()
    beta_dcov, _, _ = MAVE1(X.T, ygram, initial)
    mave1_time = time.time() - start_time
    print(f"MAVE1 time: {mave1_time:.4f} seconds")
    
    b2 = beta_dcov

    # Compute errors
    error2 = linalg_norm(b2@pinv(b2.T@b2)@b2.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsD[repeat] =error2

    error3 = linalg_norm(b3@pinv(b3.T@b3)@b3.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsF[repeat] = error3

    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    errorsG[repeat] = error1

  

# Compute means and standard deviations
mean_errorG = np.mean(errorsG)
sd_errorG = np.std(errorsG)

mean_errorD = np.mean(errorsD)
sd_errorD = np.std(errorsD)

mean_errorF = np.mean(errorsF)
sd_errorF = np.std(errorsF)

# Print the results
print('Model2-a,(200,10)')
print(f"Error  (G-WIRE): Mean = {mean_errorG:.4f}, SD = {sd_errorG:.4f}")
print(f"Error  (DCOV): Mean = {mean_errorD:.4f}, SD = {sd_errorD:.4f}")
print(f"Error  (FOPG): Mean = {mean_errorF:.4f}, SD = {sd_errorF:.4f}")


GWIRE time: 3.3564 seconds
FOPG time: 0.2137 seconds
MAVE1 time: 0.0400 seconds
Model2-a,(200,10)
Error  (G-WIRE): Mean = 1.3512, SD = 0.0000
Error  (DCOV): Mean = 0.2211, SD = 0.0000
Error  (FOPG): Mean = 1.3608, SD = 0.0000


In [7]:
import numpy as np
from scipy.linalg import norm as linalg_norm
from sklearn.covariance import GraphicalLassoCV
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from numpy.linalg import pinv




# Parameters
n = 400
p = 20
d_0 = 2
rho = 0
non_ellip = 0
num_repeats = 100
metric = "Frobenius"
model = 'model2'
eps = 0.5
neigh = False
beta_1 = np.concatenate([np.array([1, 1]), np.zeros(p - 2)])
beta_2 = np.concatenate([np.zeros(p - 2), np.array([1, 1])])
beta = np.vstack([beta_1,beta_2])
beta_true = beta.T

# To store errors across all simulations
errorsG = np.ones(num_repeats)
errorsD = np.ones(num_repeats)
errorsF = np.ones(num_repeats)
Time1 = np.zeros(num_repeats)
Time2 = np.zeros(num_repeats)
Time3 = np.zeros(num_repeats)


# Repeat simulation num_repeats times
for repeat in range(num_repeats):
    X, Y = gendata_cov(n, p, rho, non_ellip, model, eps)
    Y = np.real(Y)
    Nb = []

    if not neigh:
        # Neighborhood is unknown
        graphical_lasso_model = GraphicalLassoCV()
        graphical_lasso_model.fit(X)
        omega = graphical_lasso_model.precision_
        np.where(np.sum(omega != 0, axis=1) > 1)[0]

    for j in range(p):
        Ni = (np.nonzero(omega[j, :])[0]).tolist()
        Nb.append(Ni)

    # Gram matrix calculations
    ygram = gram_matrix(Y, 1000)
    start_time = time.time()
    beta_gwire, _ = gwire_cv(X, Y, Nb, metric, d_0, fold=5)
    gwire_time = time.time() - start_time
    Time1[repeat] = gwire_time
    b1 = beta_gwire
    ygram2 = gram_matrix(Y, 1)
    start_time = time.time()
    beta_fopg = FOPG(X, ygram2, d_0)
    fopg_time = time.time() - start_time
    Time2[repeat] = fopg_time
    b3 = beta_fopg
    initial = beta_fopg
    start_time = time.time()
    beta_dcov, _, _ = MAVE1(X.T, ygram, initial)
    mave1_time = time.time() - start_time
    Time3[repeat] = mave1_time
    
    b2 = beta_dcov

    # Compute errors
    error2 = linalg_norm(b2@pinv(b2.T@b2)@b2.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsD[repeat] =error2

    error3 = linalg_norm(b3@pinv(b3.T@b3)@b3.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsF[repeat] = error3

    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    errorsG[repeat] = error1

  

# Compute means and standard deviations
mean_errorG = np.mean(errorsG)
sd_errorG = np.std(errorsG)

mean_errorD = np.mean(errorsD)
sd_errorD = np.std(errorsD)

mean_errorF = np.mean(errorsF)
sd_errorF = np.std(errorsF)

meantimeG = np.mean(Time1)
sdtimeG = np.std(Time1)
meantimeF = np.mean(Time2)
sdtimeF = np.std(Time2)
meantimeD = np.mean(Time3)
sdtimeD = np.std(Time3)
print("GWIRE mean Time:", meantimeG)
print("GWIRE std of Time:", sdtimeG)
print("DCOV mean Time",meantimeD)
print("DCOV std of Time:", sdtimeD)
print("FOPG mean Time:",meantimeF)
print("FOPG std of Time:",sdtimeF)

# Print the results
print('Model2-a,(400,20)')
print(f"Error  (G-WIRE): Mean = {mean_errorG:.4f}, SD = {sd_errorG:.4f}")
print(f"Error  (DCOV): Mean = {mean_errorD:.4f}, SD = {sd_errorD:.4f}")
print(f"Error  (FOPG): Mean = {mean_errorF:.4f}, SD = {sd_errorF:.4f}")


GWIRE mean Time: 6.165621335506439
GWIRE std of Time: 3.142039811514146
DCOV mean Time 0.28810654163360594
DCOV std of Time: 0.38193170280901284
FOPG mean Time: 2.8821985054016115
FOPG std of Time: 0.1256975467049168
Model2-a,(400,20)
Error  (G-WIRE): Mean = 1.4056, SD = 0.0569
Error  (DCOV): Mean = 0.4322, SD = 0.3483
Error  (FOPG): Mean = 0.6153, SD = 0.4514


In [8]:
import numpy as np
from scipy.linalg import norm as linalg_norm
from sklearn.covariance import GraphicalLassoCV
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from numpy.linalg import pinv




# Parameters
n = 200
p = 10
d_0 = 2
rho = 0.5
non_ellip = 1
num_repeats = 1
metric = "Frobenius"
model = 'model2'
eps = 0.5
neigh = False
beta_1 = np.concatenate([np.array([1, 1]), np.zeros(p - 2)])
beta_2 = np.concatenate([np.zeros(p - 2), np.array([1, 1])])
beta = np.vstack([beta_1,beta_2])
beta_true = beta.T

# To store errors across all simulations
errorsG = np.ones(num_repeats)
errorsD = np.ones(num_repeats)
errorsF = np.ones(num_repeats)

# Repeat simulation num_repeats times
for repeat in range(num_repeats):
    X, Y = gendata_cov(n, p, rho, non_ellip, model, eps)
    Y = np.real(Y)
    Nb = []

    if not neigh:
        # Neighborhood is unknown
        graphical_lasso_model = GraphicalLassoCV()
        graphical_lasso_model.fit(X)
        omega = graphical_lasso_model.precision_
        np.where(np.sum(omega != 0, axis=1) > 1)[0]

    for j in range(p):
        Ni = (np.nonzero(omega[j, :])[0]).tolist()
        Nb.append(Ni)

    # Gram matrix calculations
    ygram = gram_matrix(Y, 1000)
    start_time = time.time()
    beta_gwire, _ = gwire_cv(X, Y, Nb, metric, d_0, fold=5)
    gwire_time = time.time() - start_time
    print(f"GWIRE time: {gwire_time:.4f} seconds")
    b1 = beta_gwire
    ygram2 = gram_matrix(Y, 1)
    start_time = time.time()
    beta_fopg = FOPG(X, ygram2, d_0)
    fopg_time = time.time() - start_time
    print(f"FOPG time: {fopg_time:.4f} seconds")
    b3 = beta_fopg
    initial = beta_fopg
    start_time = time.time()
    beta_dcov, _, _ = MAVE1(X.T, ygram, initial)
    mave1_time = time.time() - start_time
    print(f"MAVE1 time: {mave1_time:.4f} seconds")
    
    b2 = beta_dcov

    # Compute errors
    error2 = linalg_norm(b2@pinv(b2.T@b2)@b2.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsD[repeat] =error2

    error3 = linalg_norm(b3@pinv(b3.T@b3)@b3.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsF[repeat] = error3

    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    errorsG[repeat] = error1

  

# Compute means and standard deviations
mean_errorG = np.mean(errorsG)
sd_errorG = np.std(errorsG)

mean_errorD = np.mean(errorsD)
sd_errorD = np.std(errorsD)

mean_errorF = np.mean(errorsF)
sd_errorF = np.std(errorsF)

# Print the results
print('Model2-b,(200,10)')
print(f"Error  (G-WIRE): Mean = {mean_errorG:.4f}, SD = {sd_errorG:.4f}")
print(f"Error  (DCOV): Mean = {mean_errorD:.4f}, SD = {sd_errorD:.4f}")
print(f"Error  (FOPG): Mean = {mean_errorF:.4f}, SD = {sd_errorF:.4f}")


GWIRE time: 5.8878 seconds
FOPG time: 0.2268 seconds
MAVE1 time: 0.0250 seconds
Model2-b,(200,10)
Error  (G-WIRE): Mean = 0.7205, SD = 0.0000
Error  (DCOV): Mean = 0.4986, SD = 0.0000
Error  (FOPG): Mean = 0.4331, SD = 0.0000


In [9]:
import numpy as np
from scipy.linalg import norm as linalg_norm
from sklearn.covariance import GraphicalLassoCV
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from numpy.linalg import pinv




# Parameters
n = 400
p = 20
d_0 = 2
rho = 0.5
non_ellip = 1
num_repeats = 100
metric = "Frobenius"
model = 'model2'
eps = 0.5
neigh = False
beta_1 = np.concatenate([np.array([1, 1]), np.zeros(p - 2)])
beta_2 = np.concatenate([np.zeros(p - 2), np.array([1, 1])])
beta = np.vstack([beta_1,beta_2])
beta_true = beta.T

# To store errors across all simulations
errorsG = np.ones(num_repeats)
errorsD = np.ones(num_repeats)
errorsF = np.ones(num_repeats)
Time1 = np.zeros(num_repeats)
Time2 = np.zeros(num_repeats)
Time3 = np.zeros(num_repeats)

# Repeat simulation num_repeats times
for repeat in range(num_repeats):
    X, Y = gendata_cov(n, p, rho, non_ellip, model, eps)
    Y = np.real(Y)
    Nb = []

    if not neigh:
        # Neighborhood is unknown
        graphical_lasso_model = GraphicalLassoCV()
        graphical_lasso_model.fit(X)
        omega = graphical_lasso_model.precision_
        np.where(np.sum(omega != 0, axis=1) > 1)[0]

    for j in range(p):
        Ni = (np.nonzero(omega[j, :])[0]).tolist()
        Nb.append(Ni)

    # Gram matrix calculations
    ygram = gram_matrix(Y, 1000)
    start_time = time.time()
    beta_gwire, _ = gwire_cv(X, Y, Nb, metric, d_0, fold=5)
    gwire_time = time.time() - start_time
    Time1[repeat] = gwire_time
    b1 = beta_gwire
    ygram2 = gram_matrix(Y, 1)
    start_time = time.time()
    beta_fopg = FOPG(X, ygram2, d_0)
    fopg_time = time.time() - start_time
    Time2[repeat] = fopg_time
    b3 = beta_fopg
    initial = beta_fopg
    start_time = time.time()
    beta_dcov, _, _ = MAVE1(X.T, ygram, initial)
    mave1_time = time.time() - start_time
    Time3[repeat] = mave1_time
    
    b2 = beta_dcov

    # Compute errors
    error2 = linalg_norm(b2@pinv(b2.T@b2)@b2.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsD[repeat] =error2

    error3 = linalg_norm(b3@pinv(b3.T@b3)@b3.T-beta_true@pinv(beta_true.T@beta_true)@beta_true.T)
    errorsF[repeat] = error3

    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    errorsG[repeat] = error1

  

# Compute means and standard deviations
mean_errorG = np.mean(errorsG)
sd_errorG = np.std(errorsG)

mean_errorD = np.mean(errorsD)
sd_errorD = np.std(errorsD)

mean_errorF = np.mean(errorsF)
sd_errorF = np.std(errorsF)

meantimeG = np.mean(Time1)
sdtimeG = np.std(Time1)
meantimeF = np.mean(Time2)
sdtimeF = np.std(Time2)
meantimeD = np.mean(Time3)
sdtimeD = np.std(Time3)
print("GWIRE mean Time:", meantimeG)
print("GWIRE std of Time:", sdtimeG)
print("DCOV mean Time",meantimeD)
print("DCOV std of Time:", sdtimeD)
print("FOPG mean Time:",meantimeF)
print("FOPG std of Time:",sdtimeF)

# Print the results
print('Model2-b,(400,20)')
print(f"Error  (G-WIRE): Mean = {mean_errorG:.4f}, SD = {sd_errorG:.4f}")
print(f"Error  (DCOV): Mean = {mean_errorD:.4f}, SD = {sd_errorD:.4f}")
print(f"Error  (FOPG): Mean = {mean_errorF:.4f}, SD = {sd_errorF:.4f}")


GWIRE mean Time: 15.455653076171876
GWIRE std of Time: 0.4715376956761859
DCOV mean Time 0.13639042854309083
DCOV std of Time: 0.04344471000350054
FOPG mean Time: 2.7937469363212584
FOPG std of Time: 0.08703073291026306
Model2-b,(400,20)
Error  (G-WIRE): Mean = 0.6886, SD = 0.1576
Error  (DCOV): Mean = 0.4681, SD = 0.0781
Error  (FOPG): Mean = 0.7653, SD = 0.3366
